In [1]:
import re
import numpy as np
import pandas as pd

from collections import Counter


# Preprocess the data

In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/nlp/apps/autocorrect

/content/drive/MyDrive/Colab Notebooks/nlp/apps/autocorrect


In [3]:
path = 'data/sherlock_novels.txt'

In [4]:
def preprocessing(filename):
    """
    Takes a txt file and returns the words in the file
    Args:
        filename: path to the file
    return:
        words: list containing the words in the file
    """
    words = []
    with open(filename) as f:
        text = f.read()

    # \w: Returns a match where the string contains any word characters 
    # +: One or more occurrences
    words = re.findall(r'\w+', text.lower())
    return words


In [5]:
def get_count(words):
    """
    Returns a dict with the count of each word in a corpus
    Args:
        words: list
    return:
        word_count: dict
    """
    word_count = Counter(words)

    return word_count


# Calculate the probability of each word

Given the dictionary of word_count, compute the probability that each word will appear if randomly selected from the corpus of words.

In [6]:
def probabilities(word_count):
    """
    Returs a dict with the probability of each word
    Args:
        word_count: dict
    returns:
        probs: dict
    """
    probs = dict()
    total = sum(word_count.values())
    for word, count in word_count.items():
        probs[word] = count / total
    
    return probs


# Add the edit functions

- delete_letter
- swithc_letter
- replace_letter
- insert_letter

In [7]:
def delete_letter(word):
    """
    Takes a string and returns all the possible strings
    if we delete 1 letter from the word
    Args:
        word: str
    returns:
        del_list: list
    """
    # given the word sherlock it returns:
    # [('', 'sherlock'), ('s', 'herlock'), ('sh', 'erlock'), ('she', 'rlock'), 
    # ('sher', 'lock'), ('sherl', 'ock'), ('sherlo', 'ck'), ('sherloc', 'k')]
    split_list = [(word[:i], word[i:]) for i in range(len(word))]

    # given the word sherlock it returns
    # ['herlock', 'serlock', 'shrlock', 'shelock', 'sherock', 'sherlck', 'sherlok', 'sherloc']
    del_list = [start + remaining[1:] for start, remaining in split_list]
    
    return del_list

def switch_letter(word):
    """
    Takes a word and returns all the possible strings
    with one adjacent character switched
    Args:
        word: str
    returns:
        switches: list
    """
    split_list = [(word[:i], word[i:]) for i in range(len(word))]
    # given the word sherlock it returns
    # ['hserlock', 'sehrlock', 'shrelock', 'shelrock', 'sherolck', 'sherlcok', 'sherlokc']
    switches = [start + remaining[1] + remaining[0] + remaining[2:] for start, remaining in split_list if len(remaining) > 1]
    
    return switches


In [8]:
def replace_letter(word):
    """
    Takes a word and returns all the possible strings
    where we replaced one letter from the original word
    Args:
        word: str
    returns:
        replace_list = list
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_list = [(word[:i], word[i:]) for i in range(len(word))]
    
    # given the word sherlock it returns
    # ['aherlock', 'bherlock', 'cherlock', 'dherlock', 'eherlock', 'fherlock', 'gherlock', 'hherlock', 'iherlock', 'jherlock', .... ]
    replace_list = [start + char + remaining[1:] for start, remaining in split_list for char in letters]
    replace_list = sorted(list(replace_list))

    return replace_list

def insert_letter(word):
    """
    Takes a word and returns a list of all possible strings with
    one letter inserted at every offset
    Args:
        word: str
    returns:
        inserts: list
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_list = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    # given the word sherlock it returns
    # 'asherlock', 'bsherlock', 'csherlock', 'dsherlock', .... ]
    inserts = [start + char + remaining for start, remaining in split_list for char in letters]

    return inserts


# Combine the edits

- edit_one(): This function is going to get all the possible edits that are one edit away from a word
- edit_two(): This function is going to get all the possible edits that are two edits away from a word

In [9]:
def edit_one(word, allow_switches=True):
    """
    Takes a word and applies all the different one edits
    Args:
        word: str
        allow_switches: bool. switch edit is not very common, so we leave
            it as an option
    returns:
        edits: set. To avoid repeated words
    """
    deletes = delete_letter(word)
    inserts = insert_letter(word)
    replaces = replace_letter(word)
    if allow_switches:
        switches = switch_letter(word)
        edits = set(deletes + inserts + replaces + switches)
    else:
        edits = set(deletes + inserts + replaces)

    return edits

def edit_two(word, allow_switches=True):
    """
    Takes a word and applies all the different two edits
    Args:
        word: str
        allow_switches: bool. switch edit is not very common, so we leave
            it as an option
    returns:
        edits_2: set. To avoid repeated words
    """

    edits = [edit_2 for edit_1 in edit_one(word) for edit_2 in edit_one(edit_1)]
    edits_2 = set(edits)

    return edits_2


# Suggestions

Now, we can calculate all the possible edits with  edit_one or edit_two of a word. We will then use those strings to get the most probable word we meant to type.

In [10]:
def corrections(word, probs, vocab, n=2):
    """
    Returns the most n corrected words and their probabilities
    Args:
        word: str
        probs: dict. maps each word with its probability in the corpus
        vocab: set. Contains all the vocabilary
        n: int. number of possible word corrections we want returned in the dictionary
    returns:
        n_best: list
    """
    suggestions = []
    if word in vocab:
        suggestions.append(word)
    
    edit_1 = edit_one(word)
    edit_2 = edit_two(word)
    suggestions = edit_1.intersection(vocab) or edit_2.intersection(vocab) or vocab

    best_words = dict()
    for word in suggestions:
        best_words[word] = probs[word]
    
    best_words = Counter(best_words)
    n_best = best_words.most_common(n)

    print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best


# Minimum edit distance

This is what is going to help us to evaluate the similarity between two strings.

In [15]:
def min_edit_distance(source, target, insert_cost=1, delete_cost=1, replace_cost=2):
    """
    It calculates the minimum edit distance between two stings
    Args:
        source: str. the string to start
        target: str. string we want to end with
        insert_cost: int. edit cost
        delete_cost: int.
        replace_cost: int
    returns:
        matrix: np.array. matrix of len(source)+1 by len(target)+1 contaning the min edit distance
        med: int. the value of the minimum edit distance
    """

    m = len(source)
    n = len(target)
    matrix = np.zeros((m+1, n+1), dtype=int)

    # The first row is 0 to n
    for row in range(1, m + 1):
        matrix[row, 0] = row
    
    # the first column is 0 to n
    for col in range(1, n + 1):
        matrix[0, col] = col

    for row in range(1, m + 1):
        for col in range(1, n + 1):
            # initialize replace_cost
            r_cost = replace_cost

            # checks row character at previous row against
            # target character at previous column
            if source[row - 1] == target[col - 1]:
                # if source and target are the same r_cost is 0
                r_cost = 0

            # Update the cost at row, col based on previous entries in the cost matrix
            matrix[row, col] = min([matrix[row-1, col] + delete_cost, matrix[row, col-1] + insert_cost, matrix[row-1, col-1] + r_cost])
    
    # Set the minimum edit distance with the cost found at row m, column n
    med = matrix[m, n]

    return matrix, med
            
